## Data Processes: Second Assignment (first part)

### Grupo:
Víctor Morcuende Castell, 47315589N

Guillermo Nájera Lavid, 70845359T

Javier Rocamora García, 20081979N

Antonio Ruiz García, 06601574E

In [7]:
import pandas as pd
from sklearn import preprocessing
from collections import defaultdict
from sklearn.impute import SimpleImputer
import math

In [8]:
# Auxiliary functions

def diagnoseDate_to_ageDiagnosed(birthDate, diagnoseDate):
    age = pd.to_datetime(diagnoseDate).year-pd.to_datetime(birthDate).year
    return age

def deathDate_to_age(birthDate, deathDate):
    deathAge = pd.to_datetime(deathDate).year-pd.to_datetime(birthDate).year
    if math.isnan(deathAge):
        deathAge = 0
    return deathAge
def deathDate_to_survivalTime(diagnosisDate, deathDate):
    survivalTime = pd.to_datetime(deathDate).year-pd.to_datetime(diagnosisDate).year
    if math.isnan(survivalTime) or survivalTime < 0:
        survivalTime = -1
    return survivalTime

In [9]:
df = pd.read_excel("breast_cancer_data.xlsx", index_col='ehr')

# Deleting unused column and duplicated data
df.pop('Unnamed: 0')
df = df.drop_duplicates(keep='first')

# Duplicating the DataFrame in order to obtain the numerical variables
df_num = pd.DataFrame(data=df, columns=df.columns, index=df.index)
df_num.pop('side')
df_num.pop('neoadjuvant')
df_num.pop('grade')
df_num.pop('invasive')
df_num.pop('er_positive')
df_num.pop('pr_positive')
df_num.pop('her2_positive')
df_num.pop('hist_type')

# Dividing the DataFrame into categorical and numerical variables
num_cols = df_num.columns.tolist()
df_cat = df.drop(num_cols, axis=1)

We delete the NULL values by using the Simple Imputer

In [10]:
df_cat.side = df_cat.side.apply(lambda x: 'unknown' if (x != 'left' and x != 'right') else x)
df_cat.invasive = df_cat.invasive.apply(lambda x: 0 if x != 1 else x)

# Imputation of nulls in categorical columns using Simple Imputer
imp_cat = SimpleImputer(strategy='most_frequent')
columns = df_cat.columns
index = df_cat.index
df_cat = pd.DataFrame(imp_cat.fit_transform(df_cat), columns=columns, index=index)

# Transforming categorical values into numerical variables
df_cat.neoadjuvant = df_cat.neoadjuvant.apply(lambda x: 0.0 if x == 'no' else 1.0)

Now we are about to subsitute the categorical labels that are represented by strings with numerical values, in order to avoid working with Strings.

In [11]:
# Taking some variables out as they are already converted into numerical values
df_aux = pd.DataFrame(data=df_cat, columns=df_cat.columns, index=df_cat.index)
df_cat.pop('neoadjuvant')
df_cat.pop('invasive')
df_cat.pop('er_positive')
df_cat.pop('pr_positive')
df_cat.pop('her2_positive')
num_cols = df_cat.columns.tolist()
df_aux = df_aux.drop(num_cols, axis=1)

# Using OneHotEncoder
ohe = preprocessing.OneHotEncoder(sparse=False)
df_cat_ohe = pd.DataFrame(ohe.fit_transform(df_cat), 
                          columns=ohe.get_feature_names_out(df_cat.columns.tolist()),
                          index=df_cat.index)

# Merge both DataFrames (df_cat_ohe and df_aux)
df_cat_def = df_cat_ohe.join(other=df_aux)

We change the value of the birth_date column to the age of each patient by applying the above declared function to the dataframe, this way we will be able to know the age of each patient at the time they were diagnosed

In [12]:
# Age at which the patient was diagnosed
ageDiagnosed = pd.Series(df_num.apply(lambda x: deathDate_to_age(x.birth_date, x.diagnosis_date), axis=1), name='age_diagnosed')

# Age at which the patient died, if so
ageDead = pd.Series(df_num.apply(lambda x: deathDate_to_age(x.birth_date, x.death_date), axis=1), name='age_dead')

# Time of survival since diagnosis, 100 in case of full recovery
survivalTime = pd.Series(df_num.apply(lambda x: deathDate_to_survivalTime(x.diagnosis_date, x.death_date), axis=1), name='survival_time')

# Changing variables
df_num.pop('birth_date')
df_num.pop('diagnosis_date')
df_num.pop('death_date')
df_num = pd.merge(left=df_num, right=ageDiagnosed, on='ehr')
df_num = pd.merge(left=df_num, right=ageDead, on='ehr')
df_num = pd.merge(left=df_num, right=survivalTime, on='ehr')
print(df_num)

      ki67  recurrence_year  menarche_age  menopause_age  pregnancy  abort  \
ehr                                                                          
6849   NaN              NaN          17.0           51.0        2.0    0.0   
268    NaN              NaN          12.0            NaN        2.0    0.0   
1458   0.0              NaN          11.0            NaN        2.0    0.0   
2013  17.0              NaN           NaN            NaN        NaN    NaN   
1350  44.0              NaN          14.0            NaN        3.0    NaN   
...    ...              ...           ...            ...        ...    ...   
191   10.0              NaN          13.0           53.0        2.0    0.0   
6482   1.0              NaN          11.0           53.0        NaN    NaN   
2564  20.0              NaN          11.0            NaN        0.0    0.0   
2730  20.0              NaN           NaN           48.0        0.0    0.0   
2376  10.0              NaN          14.0            NaN        

Now we are going to eliminate the labels that do not give relevant information but would increase the computational cost of our data treatments.
e.g: Columns with too many different values, such as descriptions or names.